# Практическая работа

# Задача

Один из способов повысить эффективность взаимодействия банка с клиентами — отправлять предложение о новой услуге не всем клиентам, а только некоторым, которые выбираются по принципу наибольшей склонности к отклику на это предложение.

Задача заключается в том, чтобы предложить алгоритм, который будет выдавать склонность клиента к положительному или отрицательному отклику на предложение банка. Предполагается, что, получив такие оценки для некоторого множества клиентов, банк обратится с предложением только к тем, от кого ожидается положительный отклик.


Для решения этой задачи загрузите файлы из базы в Postgres.
Эта БД хранит информацию о клиентах банка и их персональные данные, такие как пол, количество детей и другие.

Описание таблиц с данными представлено ниже.


**D_work**

Описание статусов относительно работы:
- ID — идентификатор социального статуса клиента относительно работы;
- COMMENT — расшифровка статуса.


**D_pens**

Описание статусов относительно пенсии:
- ID — идентификатор социального статуса;
- COMMENT — расшифровка статуса.


**D_clients**

Описание данных клиентов:
- ID — идентификатор записи;
- AGE	— возраст клиента;
- GENDER — пол клиента (1 — мужчина, 0 — женщина);
- EDUCATION — образование;
- MARITAL_STATUS — семейное положение;
- CHILD_TOTAL	— количество детей клиента;
- DEPENDANTS — количество иждивенцев клиента;
- SOCSTATUS_WORK_FL	— социальный статус клиента относительно работы (1 — работает, 0 — не работает);
- SOCSTATUS_PENS_FL	— социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
- REG_ADDRESS_PROVINCE — область регистрации клиента;
- FACT_ADDRESS_PROVINCE — область фактического пребывания клиента;
- POSTAL_ADDRESS_PROVINCE — почтовый адрес области;
- FL_PRESENCE_FL — наличие в собственности квартиры (1 — есть, 0 — нет);
- OWN_AUTO — количество автомобилей в собственности.


**D_agreement**

Таблица с зафиксированными откликами клиентов на предложения банка:
- AGREEMENT_RK — уникальный идентификатор объекта в выборке;
- ID_CLIENT — идентификатор клиента;
- TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было).
    
    
**D_job**

Описание информации о работе клиентов:
- GEN_INDUSTRY — отрасль работы клиента;
- GEN_TITLE — должность;
- JOB_DIR — направление деятельности внутри компании;
- WORK_TIME — время работы на текущем месте (в месяцах);
- ID_CLIENT — идентификатор клиента.


**D_salary**

Описание информации о заработной плате клиентов:
- ID_CLIENT — идентификатор клиента;
- FAMILY_INCOME — семейный доход (несколько категорий);
- PERSONAL_INCOME — личный доход клиента (в рублях).


**D_last_credit**

Информация о последнем займе клиента:
- ID_CLIENT — идентификатор клиента;
- CREDIT — сумма последнего кредита клиента (в рублях);
- TERM — срок кредита;
- FST_PAYMENT — первоначальный взнос (в рублях).


**D_loan**

Информация о кредитной истории клиента:
- ID_CLIENT — идентификатор клиента;
- ID_LOAN — идентификатор кредита.

**D_close_loan**

Информация о статусах кредита (ссуд):
- ID_LOAN — идентификатор кредита;
- CLOSED_FL — текущий статус кредита (1 — закрыт, 0 — не закрыт).

Ниже представлен минимальный список колонок, которые должны находиться в итоговом датасете после склейки и агрегации данных. По своему усмотрению вы можете добавить дополнительные к этим колонки.

    - AGREEMENT_RK — уникальный идентификатор объекта в выборке;
    - TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было);
    - AGE — возраст клиента;
    - SOCSTATUS_WORK_FL — социальный статус клиента относительно работы (1 — работает, 0 — не работает);
    - SOCSTATUS_PENS_FL — социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
    - GENDER — пол клиента (1 — мужчина, 0 — женщина);
    - CHILD_TOTAL — количество детей клиента;
    - DEPENDANTS — количество иждивенцев клиента;
    - PERSONAL_INCOME — личный доход клиента (в рублях);
    - LOAN_NUM_TOTAL — количество ссуд клиента;
    - LOAN_NUM_CLOSED — количество погашенных ссуд клиента.


Будьте внимательны при сборке датасета: это реальные банковские данные, в которых могут наблюдаться дубли, некорректно заполненные значения или значения, противоречащие друг другу. Для получения качественной модели необходимо предварительно очистить датасет от такой информации.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Задание 1

Соберите всю информацию о клиентах в одну таблицу, где одна строчка соответствует полной информации об одном клиенте.

In [2]:
clients_df = pd.read_csv("datasets/D_clients.csv")
close_loan = pd.read_csv("datasets/D_close_loan.csv")
job_df = pd.read_csv("datasets/D_job.csv")
last_credit_df = pd.read_csv("datasets/D_close_loan.csv")
loan_df = pd.read_csv("datasets/D_loan.csv")
pens_df = pd.read_csv("datasets/D_pens.csv")
salary_df = pd.read_csv("datasets/D_salary.csv")
target_df = pd.read_csv("datasets/D_target.csv")
work_df = pd.read_csv("datasets/D_work.csv")


In [3]:
# (close_loan == last_credit_df).all()

In [4]:
main_df = (
    target_df
        .merge(clients_df, left_on="ID_CLIENT", right_on="ID", how="left")
        .drop("ID", axis=1)
        .merge(job_df, on="ID_CLIENT", how="left")
        .merge(loan_df, on="ID_CLIENT", how="left")
        .merge(close_loan, on="ID_LOAN", how="left")
        .merge(salary_df, on="ID_CLIENT", how="left")
        .drop([ 'ID_CLIENT', 'EDUCATION',
                'MARITAL_STATUS', 'REG_ADDRESS_PROVINCE', 
                'FACT_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 
                'FL_PRESENCE_FL', 'OWN_AUTO', 'GEN_INDUSTRY',
                'GEN_TITLE', 'JOB_DIR', 'WORK_TIME', 'ID_LOAN',
                'FAMILY_INCOME'], axis=1))

In [5]:
total_df = (
    main_df
        .groupby(['AGREEMENT_RK', 'TARGET',
                'AGE', 'GENDER', 'CHILD_TOTAL',
                'DEPENDANTS', 'SOCSTATUS_WORK_FL', 
                'SOCSTATUS_PENS_FL', 'PERSONAL_INCOME'], as_index=False)
        .agg({'CLOSED_FL': "count"})
        .rename(columns={"CLOSED_FL": "LOAN_NUM_TOTAL"}))

total_df["LOAN_NUM_CLOSED"] = (
    main_df
        .groupby(['AGREEMENT_RK', 'TARGET',
                'AGE', 'GENDER', 'CHILD_TOTAL',
                'DEPENDANTS', 'SOCSTATUS_WORK_FL', 
                'SOCSTATUS_PENS_FL', 'PERSONAL_INCOME'], as_index=False)
        .agg({'CLOSED_FL': "sum"})["CLOSED_FL"])

In [6]:
total_df.to_csv("datasets/total_df.csv")

## Задание 2

При помощи инструмента Streamlit проведите разведочный анализ данных. В него может входить:

* построение графиков распределений признаков
* построение матрицы корреляций
* построение графиков зависимостей целевой переменной и признаков
* вычисление числовых характеристик распределения числовых столбцов (среднее, min, max, медиана и так далее)
* любые другие ваши идеи приветствуются!

[Пример Streamlit-приложения](https://rateyourflight.streamlit.app) с разведочным анализом, прогнозом модели и оценкой ее результатов.